# Imports

In [1]:
# Imports
# Standard Imports
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import torch
import os

# Ch 2 Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import *
from sklearn.metrics import accuracy_score

# Ch 3 Imports
import torchvision
from torchvision import transforms

# Now you can import your file just like any other library
import sys
sys.path.append('../Inside-Deep-Learning/')

from idlmam import *


In [2]:
if torch.cuda.is_available():
   device = torch.device("cuda")
   B = 32
   epochs = 20
   num_workers_data_loaders = 2
elif torch.xpu.is_available():
   device = torch.device("xpu")
   B = 32
   epochs = 20
   num_workers_data_loaders = 2
else:
   device = torch.device("cpu")
   B = 1
   epochs = 5
   num_workers_data_loaders = 1

print(device)

xpu


# 1\. Now that you know how to enlarge a tensor after pooling, you can implement a convolutional autoencoder using only the bottleneck approach. Go back to chapter 7 and reimplement a convolutional autoencoder by using two rounds of pooling in the encoder countered by two rounds of transposed convolutions in the decoder.

## Get MNIST data

In [6]:
train_data = torchvision.datasets.MNIST("./", train=True,
 transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.MNIST("./", train=True,
 transform=transforms.ToTensor(), download=True)

In [7]:
class AutoEncodeDataset(Dataset): 
    """Takes a dataset with (x, y) label pairs and converts it to (x, x) pairs. 
    This makes it easy to reuse other code"""

    def __init__(self, dataset): 
        self.dataset = dataset

    def __len__(self): 
        return len(self.dataset)

    def __getitem__(self, idx): 
        x, y = self.dataset.__getitem__(idx) 
        return x, x

In [8]:
train_data_xy = train_data
train_data_xx = AutoEncodeDataset(train_data)

test_data_xy = test_data
test_data_xx = AutoEncodeDataset(test_data)


train_loader = DataLoader(train_data_xx, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data_xx, batch_size=256)

D = 28*28
C = 1

In [9]:
D = 28*28
n = 2
C = 1
classes = 10

In [10]:
len(train_data), len(train_data[0]), 

(60000, 2)

## Define Model

In [11]:
def ConvLayer(c_in, c_out=None, filter_size=3):
   if c_out is None:
        c_out = c_in

   layer = nn.Sequential(
      nn.Conv2d(c_in, c_out, filter_size, padding=filter_size//2),
      nn.Tanh(),
   )

   return layer

In [ ]:
n_filters = 32

segmentation_model2 = nn.Sequential( 
    ConvLayer(C, n_filters),
    ConvLayer(n_filters, n_filters), 

    nn.MaxPool2d(2),
    ConvLayer(n_filters, 2*n_filters), 
    ConvLayer(2*n_filters, 2*n_filters),

    nn.MaxPool2d(2),
    ConvLayer(2*n_filters, 4*n_filters), 
    ConvLayer(4*n_filters, 4*n_filters), 

    nn.ConvTranspose2d(4*n_filters, 2*n_filters, (3,3),
                       padding=1, output_padding=1, stride=2), 
    nn.BatchNorm2d(2*n_filters), 
    nn.LeakyReLU(),
    ConvLayer(2*n_filters, 2*n_filters),
    
    nn.ConvTranspose2d(2*n_filters, n_filters, (3,3),
                       padding=1, output_padding=1, stride=2), 
    nn.BatchNorm2d(n_filters), 
    nn.LeakyReLU(), 

    ConvLayer(n_filters, n_filters),
    nn.Conv2d(n_filters, 1, (3,3), padding=1),
)


In [13]:
n_filters = 32

auto_encoder = nn.Sequential(
    ConvLayer(C, n_filters),
    ConvLayer(n_filters, n_filters), 

    nn.MaxPool2d(2),
    ConvLayer(n_filters, 2*n_filters), 
    ConvLayer(2*n_filters, 2*n_filters),

    nn.MaxPool2d(2),
    ConvLayer(2*n_filters, 4*n_filters), 
    ConvLayer(4*n_filters, 4*n_filters)
)

# The decoder is a convolutional network
auto_decoder = nn.Sequential(
    nn.ConvTranspose2d(4*n_filters, 2*n_filters, (3,3),
                       padding=1, output_padding=1, stride=2), 
    nn.BatchNorm2d(2*n_filters), 
    nn.LeakyReLU(),
    ConvLayer(2*n_filters, 2*n_filters),
    
    nn.ConvTranspose2d(2*n_filters, n_filters, (3,3),
                       padding=1, output_padding=1, stride=2), 
    nn.BatchNorm2d(n_filters), 
    nn.LeakyReLU(), 

    nn.Conv2d(n_filters, classes, (3,3), padding=1)
)

model = nn.Sequential(
    auto_encoder, 
    auto_decoder 
)

In [ ]:
loss = nn.MSELoss()

results_model_cnn = train_network(model,
   loss, train_loader, test_loader=test_loader,
   score_funcs={'Accuracy': accuracy_score}, device=device, epochs=4)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]C:\Users\miles\AppData\Roaming\Python\Python313\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([256, 1, 28, 28])) that is different to the input size (torch.Size([256, 10, 28, 28])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch:   0%|          | 0/4 [02:53<?, ?it/s]
c:\Users\miles\Dev\InsideDeepLearningExercises\../Inside-Deep-Learning\idlmam.py:215: SyntaxWarning: invalid escape sequence '\e'
  """Train simple neural networks


# 2\. You may have noticed that a transposed convolution can create unevenly spaced artifacts in its output, which occur in our example diagram. These are not always a problem, but you can do better. Implement your own Conv2dExpansion(n_filters_in) class that takes the following approach: first, up-sample the image using nn.Upsample to expand the tensor width and height by a factor of 2. If you are off by a pixel, use nn.ReflectionPad2d to pad the output to the desired shape. Finally, apply a normal nn.Conv2d to perform some mixing and change the number of channels. Compare this new approach with transposed convolution and see if you can identify any pros and cons.

In [ ]:
class 